In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import NoSuchElementException
import pandas
import time

#Load the chrome driver
chrome_options = Options()
chrome_options.add_argument("--user-data-dir=user-data")
driver = webdriver.Chrome(options=chrome_options, executable_path=r"C:\Users\hp\Desktop\New folder (2)\chromedriver_win32/chromedriver.exe")
count = 0

#Open WhatsApp URL in chrome browser
driver.get("https://web.whatsapp.com/")
wait = WebDriverWait(driver, 20)

#Read data from excel
excel_data = pandas.read_excel('ulk.xlsx',engine='openpyxl')
message = excel_data['Message'][0]
#Iterate excel rows till to finish
for column in excel_data['Name'].tolist():
    # Locate search box through x_path
    search_box = '//*[@id="side"]/div[1]/div/label/div/div[2]'
    person_title = wait.until(lambda driver:driver.find_element_by_xpath(search_box))

    # Clear search box if any contact number is written in it
    person_title.clear()

    # Send contact number in search box
    person_title.send_keys(str(excel_data['Contact'][count]))
    count = count + 1

    # Wait for 3 seconds to search contact number
    time.sleep(3)

    try:
        # Load error message in case unavailability of contact number
        element = driver.find_element_by_xpath('//*[@id="pane-side"]/div[1]/div/span')
    except NoSuchElementException:
        # Format the message from excel sheet
        message = message.replace('{customer_name}',"{}")
        new_message = message.format(column)
        
        person_title.send_keys(Keys.ENTER)
        actions = ActionChains(driver)
        actions.send_keys(new_message)
        actions.send_keys(Keys.ENTER)
        actions.perform()

# Close Chrome browser
driver.quit()